In [7]:
from estnltk import Text
import csv
from datetime import datetime
from tqdm import tqdm
import psycopg2

In [2]:
from taggers.drug_tagger import DrugFieldPartTagger, DrugTagger
from cda_data_cleaning.common.db_operations import create_connection
from cda_data_cleaning.common import read_config

In [3]:
part_tagger = DrugFieldPartTagger()

In [10]:
drug_tagger = DrugTagger()

In [11]:
config_file = 'egcut_epi.ini'
config = read_config(config_file)

conn = create_connection(config)
cur = conn.cursor()

In [12]:
cur.execute("""SELECT * from original.drug;""")

In [13]:
drug_rows = cur.fetchall()

In [14]:
new_rows = []
for row in tqdm(drug_rows):
    if row[3]:
        t = Text(row[3]).tag_layer(['words'])
        part_tagger.tag(t)
        drug_tagger.tag(t)

        for j in t.parsed_drug:
            date = ''
            atc_code = ''
            drug_name = ''
            active_ingredient = ''
            if j.date:
                date = j.date
            if j.atc_code:
                atc_code = j.atc_code
            if j.drug_name:
                drug_name = j.drug_name
            if j.active_ingredient:    
                active_ingredient = j.active_ingredient
            new_rows.append([row[0], row[1], row[2], date, atc_code, drug_name, active_ingredient])    

100%|██████████| 231167/231167 [1:26:01<00:00, 44.79it/s]  


In [15]:
len(new_rows)

392935

In [16]:
len(drug_rows)

231167

In [23]:
new_rows[:100]

[[231167, '13502443', 'a', '', '', '', 'alfusosiin'],
 [231166, '21890347', 'a', '', '', '', 'timolool'],
 [231166, '21890347', 'a', '', '', 'silmatilgad', ''],
 [231165, '40993241', 'a', '\t20150504\t', '', '', 'perindopriil'],
 [231165, '40993241', 'a', '', '', '', 'indapamiid'],
 [231164, '13308414', 'a', '\t20140123\t', '', '', 'telmisartaan'],
 [231164, '13308414', 'a', '\t20140123\t', '', '', 'etorikoksiib'],
 [231164, '13308414', 'a', '\t20140130\t', '', '', 'allopurinool'],
 [231163, '42791154', 'a', '\t20150619\t', '', '', 'glükoosamiin'],
 [231162, '9992270', 'a', '', 'D07XA81', '', ''],
 [231161, '9981240', 'a', '', 'N02AB03', '', ''],
 [231161, '9981240', 'a', '', 'N02AB03', '', ''],
 [231161, '9981240', 'a', '', 'A03FA01', '', ''],
 [231161, '9981240', 'a', '', 'N02AB03', '', ''],
 [231161, '9981240', 'a', '', 'N02AA01', '', ''],
 [231161, '9981240', 'a', '', 'L02BG03', '', ''],
 [231161, '9981240', 'a', '', 'N02AA01', '', ''],
 [231161, '9981240', 'a', '', 'N02AB03', '', 

In [17]:
def clean_date(date_str):
    '''
    Turn datestring into datetime object
    '''
    if date_str:
        stripped_date = date_str.strip('\t\n, ')
    else:
        return
    if len(stripped_date) == 8:
        date = datetime.strptime(stripped_date, '%Y%m%d')
        return date
    elif len(stripped_date) == 10:
        date = datetime.strptime(stripped_date, '%d.%m.%Y')
        return date
    else:
        return 

In [18]:
cur.execute("""set role egcut_epi_work_create;""")

In [42]:
cur.execute("""drop table work.drug_parsed;""")

In [19]:
cur.execute("""create table work.drug_parsed_v1 (id bigint, epi_id text, 
epi_type text, date_raw text, date date, ATC text, drug_name text, active_ingredient text);""")

In [20]:
for row in tqdm(new_rows):
    new_date = clean_date(row[3])
    cur.execute("insert into work.drug_parsed_v1 VALUES (%s, %s, %s, %s, %s, %s, %s, %s)",  (row[0], row[1], row[2], row[3], new_date, row[4], row[5], row[6]))

100%|██████████| 392935/392935 [01:37<00:00, 4010.80it/s]


In [21]:
conn.commit()
cur.close()
conn.close()